## Importing libraries and helper functions

Importing libraries

In [1]:
%load_ext autoreload
%autoreload 2
#maths
import pandas as pd
import numpy as np

#apicalls, datacleaning
import requests
import json
from datetime import date, datetime, timedelta

#mysql
import mysql.connector 
from mysql.connector import errorcode

Importing helper functions

In [2]:
from mod3_functions import inflate_adj, inflate_adj_db

Importing config

In [ ]:
import config

In [3]:
api_key = config.api_key

## API calls

### Hourly demand on electricity in NY

In [107]:
demand_url = f'http://api.eia.gov/series/?api_key={api_key}&series_id=EBA.NYIS-ALL.D.HL'

In [108]:
response = requests.get(demand_url)
rsp=response.json()

In [111]:
#overview of results
rsp

{'request': {'command': 'series', 'series_id': 'EBA.NYIS-ALL.D.HL'},
 'series': [{'series_id': 'EBA.NYIS-ALL.D.HL',
   'name': 'Demand for New York Independent System Operator (NYIS), hourly - local time',
   'units': 'megawatthours',
   'f': 'HL',
   'description': 'Timestamps follow the ISO8601 standard (https://en.wikipedia.org/wiki/ISO_8601). Hourly representations are provided in local time for the balancing authority or region.',
   'start': '20150701T01-04',
   'end': '20191004T08-04',
   'updated': '2019-10-04T09:47:16-0400',
   'data': [['20191004T08-04', 16540],
    ['20191004T07-04', 15336],
    ['20191004T06-04', 13699],
    ['20191004T05-04', 12902],
    ['20191004T04-04', 12777],
    ['20191004T03-04', 12904],
    ['20191004T02-04', 13199],
    ['20191004T01-04', 13700],
    ['20191004T00-04', 14579],
    ['20191003T23-04', 15753],
    ['20191003T22-04', 16859],
    ['20191003T21-04', 17727],
    ['20191003T20-04', 18184],
    ['20191003T19-04', 18209],
    ['20191003T18-

In [109]:
demand_dates =  pd.DataFrame(rsp["series"][0]['data'])[0].apply(lambda x: datetime.strptime(x[:-3], '%Y%m%dT%H'))

0       2019-10-04 08:00:00
1       2019-10-04 07:00:00
2       2019-10-04 06:00:00
3       2019-10-04 05:00:00
4       2019-10-04 04:00:00
5       2019-10-04 03:00:00
6       2019-10-04 02:00:00
7       2019-10-04 01:00:00
8       2019-10-04 00:00:00
9       2019-10-03 23:00:00
10      2019-10-03 22:00:00
11      2019-10-03 21:00:00
12      2019-10-03 20:00:00
13      2019-10-03 19:00:00
14      2019-10-03 18:00:00
15      2019-10-03 17:00:00
16      2019-10-03 16:00:00
17      2019-10-03 15:00:00
18      2019-10-03 14:00:00
19      2019-10-03 13:00:00
20      2019-10-03 12:00:00
21      2019-10-03 11:00:00
22      2019-10-03 10:00:00
23      2019-10-03 09:00:00
24      2019-10-03 08:00:00
25      2019-10-03 07:00:00
26      2019-10-03 06:00:00
27      2019-10-03 05:00:00
28      2019-10-03 04:00:00
29      2019-10-03 03:00:00
                ...        
37322   2015-07-02 06:00:00
37323   2015-07-02 05:00:00
37324   2015-07-02 04:00:00
37325   2015-07-02 03:00:00
37326   2015-07-02 0

In [19]:
demand_MWh = pd.DataFrame(rsp["series"][0]['data'])[1]

0        22483.0
1        22898.0
2        23369.0
3        23649.0
4        23774.0
5        23576.0
6        23292.0
7        22709.0
8        22144.0
9        21498.0
10       20706.0
11       19961.0
12       19141.0
13       17854.0
14       16062.0
15       15172.0
16       15088.0
17       15299.0
18       15696.0
19       16421.0
20       17491.0
21       18823.0
22       20138.0
23       21215.0
24       21878.0
25       21798.0
26       21982.0
27       21816.0
28       21329.0
29       20643.0
          ...   
37286    15314.0
37287    14884.0
37288    14936.0
37289    15356.0
37290    16052.0
37291    17090.0
37292    18493.0
37293    20149.0
37294    21525.0
37295    21746.0
37296    22262.0
37297    22984.0
37298    23664.0
37299    23804.0
37300    23778.0
37301    23606.0
37302    23260.0
37303    22809.0
37304    22471.0
37305    21849.0
37306    20971.0
37307    19956.0
37308    18690.0
37309    17161.0
37310    15874.0
37311    15304.0
37312    15283.0
37313    15541

In [20]:
demand_NY = pd.concat([demand_dates,demand_MWh], axis=1)
demand_NY.columns = ["Date", "Demand [MWh]"]
demand_NY

,0,1
0,2019-10-02 20:00:00,22483.0
1,2019-10-02 19:00:00,22898.0
2,2019-10-02 18:00:00,23369.0
3,2019-10-02 17:00:00,23649.0
4,2019-10-02 16:00:00,23774.0
5,2019-10-02 15:00:00,23576.0
6,2019-10-02 14:00:00,23292.0
7,2019-10-02 13:00:00,22709.0
8,2019-10-02 12:00:00,22144.0
9,2019-10-02 11:00:00,21498.0


### Average monthly electricity retail price in NY

In [110]:
#all_sectors
monthly_retail_electricity_all_url = f'http://api.eia.gov/series/?api_key={api_key}&series_id=ELEC.PRICE.NY-ALL.M'

In [90]:
response2 = requests.get(monthly_retail_electricity_all_url)
rsp2=response2.json()

In [91]:
rsp2

{'request': {'command': 'series', 'series_id': 'ELEC.PRICE.NY-ALL.M'},
 'series': [{'series_id': 'ELEC.PRICE.NY-ALL.M',
   'name': 'Average retail price of electricity : New York : all sectors : monthly',
   'units': 'cents per kilowatthour',
   'f': 'M',
   'description': 'All end-use sectors that consume electricity; All end-use sectors that consume electricity; ',
   'copyright': 'None',
   'source': 'EIA, U.S. Energy Information Administration',
   'iso3166': 'USA-NY',
   'geography': 'USA-NY',
   'start': '200101',
   'end': '201907',
   'updated': '2019-09-23T17:31:34-0400',
   'data': [['201907', 15.43],
    ['201906', 14.8],
    ['201905', 13.87],
    ['201904', 13.5],
    ['201903', 13.42],
    ['201902', 13.82],
    ['201901', 13.68],
    ['201812', 13.6],
    ['201811', 14.17],
    ['201810', 15.26],
    ['201809', 16.1],
    ['201808', 15.78],
    ['201807', 16.22],
    ['201806', 15.47],
    ['201805', 14.32],
    ['201804', 13.88],
    ['201803', 13.84],
    ['201802', 14

In [92]:
avg_price_all_dates =  pd.DataFrame(rsp2["series"][0]['data'])[0].apply(lambda x: datetime.strptime(x, '%Y%m'))

0     2019-07-01
1     2019-06-01
2     2019-05-01
3     2019-04-01
4     2019-03-01
5     2019-02-01
6     2019-01-01
7     2018-12-01
8     2018-11-01
9     2018-10-01
10    2018-09-01
11    2018-08-01
12    2018-07-01
13    2018-06-01
14    2018-05-01
15    2018-04-01
16    2018-03-01
17    2018-02-01
18    2018-01-01
19    2017-12-01
20    2017-11-01
21    2017-10-01
22    2017-09-01
23    2017-08-01
24    2017-07-01
25    2017-06-01
26    2017-05-01
27    2017-04-01
28    2017-03-01
29    2017-02-01
         ...    
193   2003-06-01
194   2003-05-01
195   2003-04-01
196   2003-03-01
197   2003-02-01
198   2003-01-01
199   2002-12-01
200   2002-11-01
201   2002-10-01
202   2002-09-01
203   2002-08-01
204   2002-07-01
205   2002-06-01
206   2002-05-01
207   2002-04-01
208   2002-03-01
209   2002-02-01
210   2002-01-01
211   2001-12-01
212   2001-11-01
213   2001-10-01
214   2001-09-01
215   2001-08-01
216   2001-07-01
217   2001-06-01
218   2001-05-01
219   2001-04-01
220   2001-03-

In [112]:
avg_price_cent_kWh = pd.DataFrame(rsp2["series"][0]['data'])[1]

In [94]:
avg_price_all_NY = pd.concat([avg_price_all_dates,avg_price_cent_kWh], axis = 1)
avg_price_all_NY

In [71]:
#residential
monthly_retail_electricity_res_url =f'http://api.eia.gov/series/?api_key={api_key}&series_id=ELEC.PRICE.NY-RES.M'

In [72]:
response3 = requests.get(monthly_retail_electricity_res_url)
rsp3 = response3.json()

In [73]:
rsp3

{'request': {'command': 'series', 'series_id': 'ELEC.PRICE.NY-RES.M'},
 'series': [{'series_id': 'ELEC.PRICE.NY-RES.M',
   'name': 'Average retail price of electricity : New York : residential : monthly',
   'units': 'cents per kilowatthour',
   'f': 'M',
   'description': 'Electricity sold to residential customers; Electricity sold to residential customers; ',
   'copyright': 'None',
   'source': 'EIA, U.S. Energy Information Administration',
   'iso3166': 'USA-NY',
   'geography': 'USA-NY',
   'start': '200101',
   'end': '201907',
   'updated': '2019-09-23T17:31:34-0400',
   'data': [['201907', 18.65],
    ['201906', 18.56],
    ['201905', 17.38],
    ['201904', 17.56],
    ['201903', 16.86],
    ['201902', 17.64],
    ['201901', 17.29],
    ['201812', 17.34],
    ['201811', 18.17],
    ['201810', 19.29],
    ['201809', 19.28],
    ['201808', 19.03],
    ['201807', 19.38],
    ['201806', 19.3],
    ['201805', 18.52],
    ['201804', 18],
    ['201803', 17.53],
    ['201802', 18.2],
 

In [76]:
avg_price_res_dates =  pd.DataFrame(rsp3["series"][0]['data'])[0].apply(lambda x: datetime.strptime(x, '%Y%m'))
avg_price_res_cent_kWh = pd.DataFrame(rsp3["series"][0]['data'])[1]
avg_price_res_NY = pd.concat([avg_price_res_dates,avg_price_res_cent_kWh], axis = 1)
avg_price_res_NY

,0,1
0,2019-07-01,18.65
1,2019-06-01,18.56
2,2019-05-01,17.38
3,2019-04-01,17.56
4,2019-03-01,16.86
5,2019-02-01,17.64
6,2019-01-01,17.29
7,2018-12-01,17.34
8,2018-11-01,18.17
9,2018-10-01,19.29


In [77]:
#commercial
monthly_retail_electricity_com_url = f'http://api.eia.gov/series/?api_key={api_key}&series_id=ELEC.PRICE.NY-COM.M'

In [78]:
response4 = requests.get(monthly_retail_electricity_com_url)
rsp4 = response4.json()
rsp4

{'request': {'command': 'series', 'series_id': 'ELEC.PRICE.NY-COM.M'},
 'series': [{'series_id': 'ELEC.PRICE.NY-COM.M',
   'name': 'Average retail price of electricity : New York : commercial : monthly',
   'units': 'cents per kilowatthour',
   'f': 'M',
   'description': 'Electricity sold to commercial customers; Electricity sold to commercial customers; ',
   'copyright': 'None',
   'source': 'EIA, U.S. Energy Information Administration',
   'iso3166': 'USA-NY',
   'geography': 'USA-NY',
   'start': '200101',
   'end': '201907',
   'updated': '2019-09-23T17:31:34-0400',
   'data': [['201907', 15.27],
    ['201906', 14.76],
    ['201905', 13.43],
    ['201904', 13.07],
    ['201903', 12.91],
    ['201902', 13.11],
    ['201901', 12.99],
    ['201812', 13.12],
    ['201811', 13.78],
    ['201810', 14.99],
    ['201809', 16.07],
    ['201808', 15.52],
    ['201807', 16.05],
    ['201806', 15.12],
    ['201805', 13.71],
    ['201804', 13.36],
    ['201803', 13.37],
    ['201802', 13.6],


In [81]:
avg_price_com_dates =  pd.DataFrame(rsp4["series"][0]['data'])[0].apply(lambda x: datetime.strptime(x, '%Y%m'))
avg_price_com_cent_kWh = pd.DataFrame(rsp4["series"][0]['data'])[1]
avg_price_com_NY = pd.concat([avg_price_com_dates,avg_price_com_cent_kWh], axis = 1)
avg_price_com_NY

,0,1
0,2019-07-01,15.27
1,2019-06-01,14.76
2,2019-05-01,13.43
3,2019-04-01,13.07
4,2019-03-01,12.91
5,2019-02-01,13.11
6,2019-01-01,12.99
7,2018-12-01,13.12
8,2018-11-01,13.78
9,2018-10-01,14.99


In [117]:
#industrial
monthly_retail_electricity_ind_url = f'http://api.eia.gov/series/?api_key={api_key}&series_id=ELEC.PRICE.NY-IND.M'

In [83]:
response5 = requests.get(monthly_retail_electricity_ind_url)
rsp5 = response5.json()
rsp5

{'request': {'command': 'series', 'series_id': 'ELEC.PRICE.NY-IND.M'},
 'series': [{'series_id': 'ELEC.PRICE.NY-IND.M',
   'name': 'Average retail price of electricity : New York : industrial : monthly',
   'units': 'cents per kilowatthour',
   'f': 'M',
   'description': 'Electricity sold to industrial customers; Electricity sold to industrial customers; ',
   'copyright': 'None',
   'source': 'EIA, U.S. Energy Information Administration',
   'iso3166': 'USA-NY',
   'geography': 'USA-NY',
   'start': '200101',
   'end': '201907',
   'updated': '2019-09-23T17:31:34-0400',
   'data': [['201907', 5.69],
    ['201906', 5.25],
    ['201905', 5.74],
    ['201904', 5.45],
    ['201903', 5.51],
    ['201902', 5.63],
    ['201901', 5.67],
    ['201812', 5.61],
    ['201811', 5.77],
    ['201810', 6.07],
    ['201809', 6.07],
    ['201808', 6.06],
    ['201807', 6.29],
    ['201806', 5.99],
    ['201805', 6.55],
    ['201804', 5.59],
    ['201803', 5.49],
    ['201802', 6.65],
    ['201801', 6.

In [84]:
avg_price_ind_dates =  pd.DataFrame(rsp5["series"][0]['data'])[0].apply(lambda x: datetime.strptime(x, '%Y%m'))
avg_price_ind_cent_kWh = pd.DataFrame(rsp5["series"][0]['data'])[1]
avg_price_ind_NY = pd.concat([avg_price_ind_dates,avg_price_ind_cent_kWh], axis = 1)
avg_price_ind_NY

,0,1
0,2019-07-01,5.69
1,2019-06-01,5.25
2,2019-05-01,5.74
3,2019-04-01,5.45
4,2019-03-01,5.51
5,2019-02-01,5.63
6,2019-01-01,5.67
7,2018-12-01,5.61
8,2018-11-01,5.77
9,2018-10-01,6.07


In [85]:
#transportation
monthly_retail_electricity_tra_url=f'http://api.eia.gov/series/?api_key={api_key}&series_id=ELEC.PRICE.NY-TRA.M'

In [86]:
response6 = requests.get(monthly_retail_electricity_tra_url)
rsp6 = response6.json()
rsp6

{'request': {'command': 'series', 'series_id': 'ELEC.PRICE.NY-TRA.M'},
 'series': [{'series_id': 'ELEC.PRICE.NY-TRA.M',
   'name': 'Average retail price of electricity : New York : transportation : monthly',
   'units': 'cents per kilowatthour',
   'f': 'M',
   'description': 'Electricity sold to transportation customers; Electricity sold to transportation customers; ',
   'copyright': 'None',
   'source': 'EIA, U.S. Energy Information Administration',
   'iso3166': 'USA-NY',
   'geography': 'USA-NY',
   'start': '200101',
   'end': '201907',
   'updated': '2019-09-23T17:31:34-0400',
   'data': [['201907', 12.85],
    ['201906', 12.95],
    ['201905', 12.36],
    ['201904', 11.88],
    ['201903', 11.67],
    ['201902', 12.24],
    ['201901', 11.96],
    ['201812', 12.1],
    ['201811', 12.04],
    ['201810', 12.08],
    ['201809', 13.07],
    ['201808', 12.05],
    ['201807', 13.34],
    ['201806', 13.65],
    ['201805', 12.27],
    ['201804', 12.11],
    ['201803', 12.28],
    ['20180

In [87]:
avg_price_tra_dates =  pd.DataFrame(rsp6["series"][0]['data'])[0].apply(lambda x: datetime.strptime(x, '%Y%m'))
avg_price_tra_cent_kWh = pd.DataFrame(rsp6["series"][0]['data'])[1]
avg_price_tra_NY = pd.concat([avg_price_tra_dates,avg_price_tra_cent_kWh], axis = 1)
avg_price_tra_NY

,0,1
0,2019-07-01,12.85
1,2019-06-01,12.95
2,2019-05-01,12.36
3,2019-04-01,11.88
4,2019-03-01,11.67
5,2019-02-01,12.24
6,2019-01-01,11.96
7,2018-12-01,12.10
8,2018-11-01,12.04
9,2018-10-01,12.08


Create dataframe from all calls

In [96]:
data_frames = [avg_price_all_NY, avg_price_com_NY,avg_price_ind_NY,avg_price_res_NY,avg_price_tra_NY]

In [97]:
from functools import reduce
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=[0],
                                            how='outer'), data_frames)

In [98]:
df_merged.columns = ['Date', 'All_sectors', 'Commercial', 'Industrial', 'Residential', 'Transportation']
df_merged

,Date,All_sectors,Commercial,Industrial,Residential,Transportation
0,2019-07-01,15.43,15.27,5.69,18.65,12.85
1,2019-06-01,14.80,14.76,5.25,18.56,12.95
2,2019-05-01,13.87,13.43,5.74,17.38,12.36
3,2019-04-01,13.50,13.07,5.45,17.56,11.88
4,2019-03-01,13.42,12.91,5.51,16.86,11.67
5,2019-02-01,13.82,13.11,5.63,17.64,12.24
6,2019-01-01,13.68,12.99,5.67,17.29,11.96
7,2018-12-01,13.60,13.12,5.61,17.34,12.10
8,2018-11-01,14.17,13.78,5.77,18.17,12.04
9,2018-10-01,15.26,14.99,6.07,19.29,12.08


Send to csv file

In [85]:
pd.DataFrame.to_csv(df_merged, 'avg_retail_prices_NY.csv', sep=',', na_rep='.', index=False)

In [88]:
pd.DataFrame.to_csv(demand_NY, 'hourly_demand_NY.csv', sep=',', na_rep='.', index=False)

### Average monthly electricity retail price in all states

In [89]:
list_of_states = ["AL", "AK", "AZ", "AR", "CA", "CO","CT", "DE", "DC", "FL", "GA",
                 "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", "MA",
                 "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY",
                 "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX",
                 "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [141]:
retail_energy_price_by_state=pd.DataFrame()
for state in list_of_states:
    url = f'http://api.eia.gov/series/?api_key={api_key}&series_id=ELEC.PRICE.{state}-ALL.M'
    response = requests.get(url)
    rsp = response.json()
    avg_price_all_dates =  pd.DataFrame(rsp["series"][0]['data'])[0].apply(lambda x: datetime.strptime(x, '%Y%m'))
    avg_price_cent_kWh = pd.DataFrame(rsp["series"][0]['data'])[1]
    retail_energy_price_by_state["Date"] = avg_price_all_dates
    retail_energy_price_by_state[state] = avg_price_cent_kWh

In [142]:
retail_energy_price_by_state

,Date,AL,AK,AZ,AR,CA,CO,CT,DE,DC,...,SD,TN,TX,UT,VT,VA,WA,WV,WI,WY
0,2019-07-01,10.45,20.84,11.46,8.55,18.63,10.61,18.24,10.38,11.80,...,10.22,10.06,8.80,8.98,15.07,9.74,7.83,8.45,11.42,8.26
1,2019-06-01,10.05,20.61,11.15,8.46,18.53,10.61,19.47,10.17,11.63,...,10.48,9.93,8.66,8.76,15.25,9.76,7.84,8.46,11.19,8.19
2,2019-05-01,10.01,20.12,10.98,7.98,16.09,9.76,19.14,10.92,12.58,...,10.02,9.43,8.41,8.27,15.10,9.56,7.82,8.30,10.94,8.13
3,2019-04-01,9.91,20.77,10.45,7.90,15.02,9.75,19.67,11.15,12.68,...,9.78,9.59,8.49,7.82,14.92,9.56,7.99,8.60,10.90,7.88
4,2019-03-01,9.62,20.21,10.03,8.06,15.60,9.92,19.29,10.87,12.14,...,9.54,9.75,8.52,8.11,15.04,9.49,8.28,8.46,10.84,8.06
5,2019-02-01,9.59,20.03,10.19,7.91,16.17,10.00,19.82,10.68,12.80,...,9.48,9.90,8.60,8.10,15.18,9.53,8.26,8.31,11.04,8.25
6,2019-01-01,9.40,19.81,10.10,7.97,15.66,9.68,19.20,10.73,12.34,...,9.34,9.86,8.53,7.89,14.89,9.50,7.99,7.94,10.86,7.90
7,2018-12-01,9.42,19.86,10.03,7.24,16.20,9.72,18.83,10.65,12.54,...,9.69,9.81,8.24,7.64,15.32,9.36,8.04,8.34,10.68,7.93
8,2018-11-01,9.58,19.20,10.07,7.25,16.55,9.91,18.57,10.86,12.66,...,9.62,9.80,8.55,7.72,15.20,9.40,8.04,8.75,10.96,7.95
9,2018-10-01,9.52,20.10,10.97,7.08,16.21,10.19,18.38,11.10,12.33,...,10.25,9.48,8.52,8.17,15.01,9.35,7.95,8.32,10.80,7.78


Create new dataframe with prices adjusted for inflation

In [151]:
retail_energy_price_by_state_adj = inflate_adj_db(retail_energy_price_by_state, "Date", list_of_states)
retail_energy_price_by_state_adj

,Date,AL,AK,AZ,AR,CA,CO,CT,DE,DC,...,SD,TN,TX,UT,VT,VA,WA,WV,WI,WY
0,2019-07-01,10.449999,20.839998,11.459999,8.549999,18.629999,10.609999,18.239999,10.379999,11.799999,...,10.219999,10.059999,8.799999,8.979999,15.069999,9.739999,7.829999,8.449999,11.419999,8.259999
1,2019-06-01,10.049999,20.609998,11.149999,8.459999,18.529999,10.609999,19.469998,10.169999,11.629999,...,10.479999,9.929999,8.659999,8.759999,15.249999,9.759999,7.839999,8.459999,11.189999,8.189999
2,2019-05-01,10.009999,20.119998,10.979999,7.979999,16.089999,9.759999,19.139999,10.919999,12.579999,...,10.019999,9.429999,8.409999,8.269999,15.099999,9.559999,7.819999,8.299999,10.939999,8.129999
3,2019-04-01,9.909999,20.769998,10.449999,7.899999,15.019999,9.749999,19.669998,11.149999,12.679999,...,9.779999,9.589999,8.489999,7.819999,14.919999,9.559999,7.989999,8.599999,10.899999,7.879999
4,2019-03-01,9.619999,20.209998,10.029999,8.059999,15.599999,9.919999,19.289998,10.869999,12.139999,...,9.539999,9.749999,8.519999,8.109999,15.039999,9.489999,8.279999,8.459999,10.839999,8.059999
5,2019-02-01,9.589999,20.029998,10.189999,7.909999,16.169999,9.999999,19.819998,10.679999,12.799999,...,9.479999,9.899999,8.599999,8.099999,15.179999,9.529999,8.259999,8.309999,11.039999,8.249999
6,2019-01-01,9.399999,19.809998,10.099999,7.969999,15.659999,9.679999,19.199999,10.729999,12.339999,...,9.339999,9.859999,8.529999,7.889999,14.889999,9.499999,7.989999,7.939999,10.859999,7.899999
7,2018-12-01,9.624494,20.291130,10.247736,7.397169,16.551677,9.931006,19.238771,10.881195,12.812224,...,9.900355,10.022960,8.418878,7.805853,15.652574,9.563191,8.214536,8.521049,10.911847,8.102148
8,2018-11-01,9.787967,19.616803,10.288604,7.407386,16.909275,10.125131,18.973126,11.095754,12.934829,...,9.828836,10.012743,8.735607,7.887589,15.529969,9.604060,8.214536,8.939949,11.197925,8.122582
9,2018-10-01,9.726665,20.536340,11.208142,7.233696,16.561894,10.411209,18.779002,11.340964,12.597666,...,10.472512,9.685796,8.704956,8.347358,15.335844,9.552974,8.122582,8.500615,11.034452,7.948892


In [104]:
inflate_adj(df_merged, "Date", "All_sectors")

,Date,All_sectors,Commercial,Industrial,Residential,Transportation,adj_Price,adjAll_sectors,adj_All_sectors
0,2019-07-01,15.43,15.27,5.69,18.65,12.85,15.429999,15.429999,15.429999
1,2019-06-01,14.80,14.76,5.25,18.56,12.95,14.799999,14.799999,14.799999
2,2019-05-01,13.87,13.43,5.74,17.38,12.36,13.869999,13.869999,13.869999
3,2019-04-01,13.50,13.07,5.45,17.56,11.88,13.499999,13.499999,13.499999
4,2019-03-01,13.42,12.91,5.51,16.86,11.67,13.419999,13.419999,13.419999
5,2019-02-01,13.82,13.11,5.63,17.64,12.24,13.819999,13.819999,13.819999
6,2019-01-01,13.68,12.99,5.67,17.29,11.96,13.679999,13.679999,13.679999
7,2018-12-01,13.60,13.12,5.61,17.34,12.10,13.895235,13.895235,13.895235
8,2018-11-01,14.17,13.78,5.77,18.17,12.04,14.477609,14.477609,14.477609
9,2018-10-01,15.26,14.99,6.07,19.29,12.08,15.591271,15.591271,15.591271


### Net generation of energy by source

In [41]:
list_of_fuels = ["COL","NG", "WAT", "OTH", "NUC", "OIL", "SUN", "WND"]

New York

In [129]:
net_gen_hr_NY=pd.DataFrame()
for fuel in list_of_fuels:
    url = f'http://api.eia.gov/series/?api_key={api_key}&series_id=EBA.NYIS-ALL.NG.{fuel}.HL'
    response = requests.get(url)
    rsp = response.json()
    net_gen_dates =  pd.DataFrame(rsp["series"][0]['data'])[0].apply(lambda x: datetime.strptime(x[:-3], '%Y%m%dT%H'))
    net_gen_MWh = pd.DataFrame(rsp["series"][0]['data'])[1]
    net_gen_hr_NY["Date"] = net_gen_dates
    net_gen_hr_NY[fuel] = net_gen_MWh

In [130]:
net_gen_hr_NY

,Date,COL,NG,WAT,OTH,OIL,SUN,WND,NUC
0,2019-10-02 00:00:00,0,4012,3088,191,922,0,765,5211
1,2019-10-01 23:00:00,0,5043,3059,190,974,0,847,5238
2,2019-10-01 22:00:00,0,6250,3179,190,979,0,1105,5253
3,2019-10-01 21:00:00,0,7106,3409,190,977,0,1250,5240
4,2019-10-01 20:00:00,0,7455,3531,191,977,0,1183,5235
5,2019-10-01 19:00:00,0,7513,3428,188,975,0,952,5222
6,2019-10-01 18:00:00,0,7504,3204,193,972,0,1121,5235
7,2019-10-01 17:00:00,0,7414,3357,198,975,0,1083,5257
8,2019-10-01 16:00:00,0,6950,3597,205,980,0,955,5283
9,2019-10-01 15:00:00,0,6600,3406,213,980,0,832,5270


In [288]:
pd.DataFrame.to_csv(net_gen_hr_NY, 'net_gen_hr_NY.csv', sep=',', na_rep='.', index=False)

California

In [131]:
net_gen_hr_CA=pd.DataFrame()
for fuel in list_of_fuels:
    url = f'http://api.eia.gov/series/?api_key={api_key}&series_id=EBA.CISO-ALL.NG.{fuel}.HL'
    response = requests.get(url)
    rsp = response.json()
    net_gen_dates =  pd.DataFrame(rsp["series"][0]['data'])[0].apply(lambda x: datetime.strptime(x[:-3], '%Y%m%dT%H'))
    net_gen_MWh = pd.DataFrame(rsp["series"][0]['data'])[1]
    net_gen_hr_CA["Date"] = net_gen_dates
    net_gen_hr_CA[fuel] = net_gen_MWh

In [132]:
net_gen_hr_CA

,Date,COL,NG,WAT,OTH,OIL,SUN,WND,NUC
0,2019-10-02 00:00:00,18,13166,1278.0,-7.0,-1,534,1475.0,1140
1,2019-10-01 23:00:00,16,12979,1277.0,-7.0,-2,534,1475.0,1142
2,2019-10-01 22:00:00,14,12168,1267.0,-17.0,-2,533,1475.0,1142
3,2019-10-01 21:00:00,14,11952,1409.0,1.0,-2,533,1475.0,1138
4,2019-10-01 20:00:00,12,11791,1182.0,-53.0,-2,604,1475.0,1137
5,2019-10-01 19:00:00,14,2908,1526.0,7.0,-2,9477,1475.0,1136
6,2019-10-01 18:00:00,14,3294,2536.0,19.0,-2,9457,1475.0,1136
7,2019-10-01 17:00:00,14,4316,3462.0,-7.0,-2,9248,1475.0,1138
8,2019-10-01 16:00:00,14,4947,3372.0,5.0,-2,9121,1641.0,1136
9,2019-10-01 15:00:00,15,4608,2952.0,20.0,-2,9990,1742.0,1138


In [289]:
pd.DataFrame.to_csv(net_gen_hr_CA, 'net_gen_hr_CA.csv', sep=',', na_rep='.', index=False)

Florida

In [42]:
net_gen_hr_FLA=pd.DataFrame()
for fuel in list_of_fuels:
    url = f'http://api.eia.gov/series/?api_key={api_key}&series_id=EBA.FLA-ALL.NG.{fuel}.HL'
    response = requests.get(url)
    rsp = response.json()
    net_gen_dates =  pd.DataFrame(rsp["series"][0]['data'])[0].apply(lambda x: datetime.strptime(x[:-3], '%Y%m%dT%H'))
    net_gen_MWh = pd.DataFrame(rsp["series"][0]['data'])[1]
    net_gen_hr_FLA["Date"] = net_gen_dates
    net_gen_hr_FLA[fuel] = net_gen_MWh

KeyError: 'series'

In [43]:
net_gen_hr_FLA

,Date,COL,NG,WAT,OTH,NUC,OIL,SUN
0,2019-10-02 00:00:00,1978,17474,14.0,2257,3804,0,0
1,2019-10-01 23:00:00,2092,19917,13.0,2304,3805,1,0
2,2019-10-01 22:00:00,2109,22559,11.0,2484,3800,0,0
3,2019-10-01 21:00:00,2146,24599,11.0,2594,3800,0,0
4,2019-10-01 20:00:00,2075,25943,11.0,2664,3801,0,1
5,2019-10-01 19:00:00,2082,26318,10.0,2621,3797,0,213
6,2019-10-01 18:00:00,2144,26983,10.0,2658,3801,36,684
7,2019-10-01 17:00:00,2176,27159,10.0,2671,3794,0,984
8,2019-10-01 16:00:00,2168,26491,10.0,2663,3797,0,1235
9,2019-10-01 15:00:00,2127,26375,10.0,2652,3799,0,1252


In [290]:
pd.DataFrame.to_csv(net_gen_hr_FLA, 'net_gen_hr_FLA.csv', sep=',', na_rep='.', index=False)

New England

In [522]:
net_gen_hr_NE=pd.DataFrame()
for fuel in list_of_fuels:
    url = f'http://api.eia.gov/series/?api_key={api_key}&series_id=EBA.NE-ALL.NG.{fuel}.HL'
    response = requests.get(url)
    rsp = response.json()
    net_gen_dates =  pd.DataFrame(rsp["series"][0]['data'])[0].apply(lambda x: datetime.strptime(x[:-3], '%Y%m%dT%H'))
    net_gen_MWh = pd.DataFrame(rsp["series"][0]['data'])[1]
    net_gen_hr_NE["Date"] = net_gen_dates
    net_gen_hr_NE[fuel] = net_gen_MWh

In [523]:
net_gen_hr_NE

,Date,COL,NG,WAT,OTH,NUC,OIL,SUN,WND
0,2019-10-02 00:00:00,0,6051,320,635,3288,0,0,646
1,2019-10-01 23:00:00,0,6978,262,632,3275,4,0,672
2,2019-10-01 22:00:00,14,8123,277,648,3273,15,0,610
3,2019-10-01 21:00:00,26,8748,433,649,3279,15,0,613
4,2019-10-01 20:00:00,31,9196,1119,640,3293,15,0,527
5,2019-10-01 19:00:00,39,9138,1212,641,3315,15,1,502
6,2019-10-01 18:00:00,26,9261,1178,630,3307,15,18,445
7,2019-10-01 17:00:00,26,9034,915,635,3325,15,47,379
8,2019-10-01 16:00:00,26,8722,588,631,3310,15,51,341
9,2019-10-01 15:00:00,26,8476,560,610,3321,15,61,246


In [524]:
pd.DataFrame.to_csv(net_gen_hr_NE, 'net_gen_hr_NE.csv', sep=',', na_rep='.', index=False)

## MySQL Database data inserting

Create tables in database

In [119]:
db_name = 'energydb'

In [473]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.login,
    passwd = config.password,
    database = db_name
)
cursor = cnx.cursor(buffered=True)

In [468]:
table_net_gen_NY = """CREATE TABLE NY_net_gen(
                    Date DATETIME PRIMARY KEY,
                    Coal REAL,
                    NaturalGas REAL,
                    Hydro REAL,
                    Other REAL,
                    Petroleum REAL,
                    Solar REAL,
                    Wind REAL,
                    Nuclear REAL)"""

In [471]:
cursor.execute(table_net_gen_NY)
cnx.commit()

In [486]:
table_net_gen_CA = """CREATE TABLE CA_net_gen(
                    Date DATETIME PRIMARY KEY,
                    Coal REAL,
                    NaturalGas REAL,
                    Hydro REAL,
                    Other REAL,
                    Petroleum REAL,
                    Solar REAL,
                    Wind REAL,
                    Nuclear REAL)"""

In [489]:
cursor.execute(table_net_gen_CA)
cnx.commit()

In [565]:
table_net_gen_FLA = """CREATE TABLE FLA_net_gen(
                    Date DATETIME PRIMARY KEY,
                    Coal REAL,
                    NaturalGas REAL,
                    Hydro REAL,
                    Other REAL,
                    Petroleum REAL,
                    Solar REAL,
                    Nuclear REAL)"""

In [568]:
cursor.execute(table_net_gen_FLA)
cnx.commit()

In [496]:
table_net_gen_NE = """CREATE TABLE NE_net_gen(
                    Date DATETIME PRIMARY KEY,
                    Coal REAL,
                    NaturalGas REAL,
                    Hydro REAL,
                    Other REAL,
                    Petroleum REAL,
                    Solar REAL,
                    Wind REAL,
                    Nuclear REAL)"""

In [527]:
cursor.execute(table_net_gen_NE)
cnx.commit()

In [125]:
table_demand_NY = """CREATE TABLE NY_demand(
                    Date DATETIME PRIMARY KEY,
                    Demand REAL)"""

In [ ]:
cursor.execute(table_demand_NY)
cnx.commit()

Inserting to database

In [ ]:
table_name = "retail_price_adj"
column_names = list_of_states
column_datatype = "REAL"
create_st='CREATE TABLE '+table_name + "(`dates` DATE PRIMARY KEY, "
i=0
while i<len(column_names)-1 :
    create_st =create_st +f"`{column_names[i]}`" +' '+column_datatype+', '
    i=i+1

create_st =create_st +f"`{column_names[i]}`"  +' '+column_datatype+');'

In [476]:
cursor.execute(create_st)
cnx.commit()

In [121]:
def send_to_sql_price(dflisty, cursor):
    """
    sends energy prices for all states to sql db
    """
    for i, rows in dflisty.iterrows():
        cursor.execute("""INSERT INTO retail_price(dates, AL , AK , AZ , AR , CA , CO , CT , DE , DC , FL , GA , HI , `ID` , IL , `IN` , IA , KS , KY , LA , ME , MD , MA , MI , MN , MS , MO , MT , NE , NV , NH , NJ , NM , NY , NC , ND , OH , OK , `OR` , PA , RI , SC , SD , TN , TX , UT , VT , VA , WA , WV , WI , WY) VALUES ('{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}',"{}")""".format(rows['Date'],rows['AL'],rows['AK'],rows['AZ'],rows['AR'],rows['CA'],rows['CO'],rows['CT'],rows['DE'],rows['DC'],rows['FL'],rows['GA'],rows['HI'],rows['ID'],rows['IL'],rows['IN'],rows['IA'],rows['KS'],rows['KY'],rows['LA'],rows['ME'],rows['MD'],rows['MA'],rows['MI'],rows['MN'],rows['MS'],rows['MO'],rows['MT'],rows['NE'],rows['NV'],rows['NH'],rows['NJ'],rows['NM'],rows['NY'],rows['NC'],rows['ND'],rows['OH'],rows['OK'],rows['OR'],rows['PA'],rows['RI'],rows['SC'],rows['SD'],rows['TN'],rows['TX'],rows['UT'],rows['VT'],rows['VA'],rows['WA'],rows['WV'],rows['WI'],rows['WY']))
        cnx.commit()

In [455]:
send_to_sql_price(retail_energy_price_by_state,cursor)

In [122]:
def send_to_sql_price_adj(dflisty, cursor):
    """
    sends energy prices for all states to sql db
    """
    for i, rows in dflisty.iterrows():
        cursor.execute("""INSERT INTO retail_price_adj(dates, AL , AK , AZ , AR , CA , CO , CT , DE , DC , FL , GA , HI , `ID` , IL , `IN` , IA , KS , KY , LA , ME , MD , MA , MI , MN , MS , MO , MT , NE , NV , NH , NJ , NM , NY , NC , ND , OH , OK , `OR` , PA , RI , SC , SD , TN , TX , UT , VT , VA , WA , WV , WI , WY) VALUES ('{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}',"{}")""".format(rows['Date'],rows['AL'],rows['AK'],rows['AZ'],rows['AR'],rows['CA'],rows['CO'],rows['CT'],rows['DE'],rows['DC'],rows['FL'],rows['GA'],rows['HI'],rows['ID'],rows['IL'],rows['IN'],rows['IA'],rows['KS'],rows['KY'],rows['LA'],rows['ME'],rows['MD'],rows['MA'],rows['MI'],rows['MN'],rows['MS'],rows['MO'],rows['MT'],rows['NE'],rows['NV'],rows['NH'],rows['NJ'],rows['NM'],rows['NY'],rows['NC'],rows['ND'],rows['OH'],rows['OK'],rows['OR'],rows['PA'],rows['RI'],rows['SC'],rows['SD'],rows['TN'],rows['TX'],rows['UT'],rows['VT'],rows['VA'],rows['WA'],rows['WV'],rows['WI'],rows['WY']))
        cnx.commit()

In [483]:
send_to_sql_price_adj(retail_energy_price_by_state_adj,cursor)

In [511]:
def send_to_sql_net_gen(dflisty, cursor,region):
    """
    sends net generation data to sql db
    """
    for i, rows in dflisty.iterrows():
        cursor.execute(f"""INSERT INTO {region}_net_gen(Date, Coal, NaturalGas, Hydro,
                    Other, Petroleum, Solar, Wind, Nuclear) VALUES ("{rows['Date']}","{rows['COL']}","{rows['NG']}","{rows['WAT']}","{rows['OTH']}","{rows['OIL']}","{rows['SUN']}","{rows['WND']}","{rows['NUC']}")""")
        cnx.commit()

In [539]:
send_to_sql_net_gen(net_gen_hr_NY,cursor, 'NY')

IntegrityError: 1062 (23000): Duplicate entry '2018-11-03 11:00:00' for key 'PRIMARY'

In [541]:
send_to_sql_net_gen(net_gen_hr_NE,cursor, 'NE')

In [559]:
send_to_sql_net_gen(net_gen_hr_CA,cursor, 'CA')

In [124]:
def send_to_sql_net_gen_fla(dflisty, cursor,region):
    """
    sends net generation data from Florida to sql db
    """
    for i, rows in dflisty.iterrows():
        cursor.execute(f"""INSERT INTO {region}_net_gen(Date, Coal, NaturalGas, Hydro,
                    Other, Petroleum, Solar, Nuclear) VALUES ("{rows['Date']}","{rows['COL']}","{rows['NG']}","{rows['WAT']}","{rows['OTH']}","{rows['OIL']}","{rows['SUN']}","{rows['NUC']}")""")
        cnx.commit()

In [47]:
send_to_sql_net_gen_fla(net_gen_hr_FLA,cursor, "FLA")

In [30]:
def send_to_sql_demand(dflisty, cursor):
    """
    sends df to sql db
    """
    for i, rows in dflisty.iterrows():
        cursor.execute(f"""INSERT INTO NY_demand(Date, Demand) VALUES ("{rows['Date']}","{rows['Demand [MWh]']}")""")
        cnx.commit()

In [39]:
send_to_sql_demand(demand_NY,cursor)

Another way to put dataframe to sql db

In [66]:
import sqlalchemy
database_username = config.login
database_password = config.password
database_ip       = config.host
database_name     = db_name
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [99]:
df_merged

,Date,All_sectors,Commercial,Industrial,Residential,Transportation
0,2019-07-01,15.43,15.27,5.69,18.65,12.85
1,2019-06-01,14.80,14.76,5.25,18.56,12.95
2,2019-05-01,13.87,13.43,5.74,17.38,12.36
3,2019-04-01,13.50,13.07,5.45,17.56,11.88
4,2019-03-01,13.42,12.91,5.51,16.86,11.67
5,2019-02-01,13.82,13.11,5.63,17.64,12.24
6,2019-01-01,13.68,12.99,5.67,17.29,11.96
7,2018-12-01,13.60,13.12,5.61,17.34,12.10
8,2018-11-01,14.17,13.78,5.77,18.17,12.04
9,2018-10-01,15.26,14.99,6.07,19.29,12.08


In [102]:
df_merged.to_sql(con=database_connection, name='NY_retail_price_by_sector', if_exists='replace')

Close all connections

In [123]:
cursor.close()
cnx.close()